In [32]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='ff2ca45c-1e03-4f70-a1e4-366c5555ede7', project_access_token='p-36349a285fcfaf72dd5ed603fda2088e50f0f7ce')
pc = project.project_context


<div style="text-align: center;margin-bottom:3em">
<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
</div>

## Libraries imported

In [142]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import lxml       
import requests   

from urllib.request import urlopen
import ssl
import csv

print('Libraries imported.')

Libraries imported.


In [143]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


## Scrapping data

In [261]:
#Current version is the hell https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

source = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695').text

soup = BeautifulSoup(source, 'lxml')
data_table = soup.find('table', {'class' : 'wikitable sortable'})

print('Scrapping data')

Scrapping data


## Create dataframe


In [269]:
df = pd.read_html(data_table.prettify(), header=None, skiprows=0)[0]
df.columns = ['Postalcode', 'Borough', 'Neighbourhood']
#df.head()
df.shape

(288, 3)

## Remove "Not Assigned"

In [270]:
indexNo = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNo, inplace = True)
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Combining into one row with the neighborhoods

In [273]:
df_t = df.groupby(['Postalcode', 'Borough'], sort = False).agg( ','.join)
df_combined = df_t.reset_index()

## Shape method to print the number of rows of your dataframe

In [274]:
df_combined.shape

(103, 3)